## Boundary matrix, reduction, output barcode (sparse matrix)

Creating simplex class and reading function

In [88]:
import numpy as np

class simplex:
    def __init__(self, line):
        self.dim, self.val, self.vert = self.parse_line(line)
    
    def parse_line(self, line):
        splited_line = line.split()
        val = float(splited_line[0])
        dim = int(splited_line[1])
        vert = [int(x) for x in splited_line[2:]]
        assert(len(vert)==dim+1)
        return dim, val, vert

def read_filtration(name):
    simplices = []
    with open(name, "r") as ins:
        for line in ins:
            simplices.append(simplex(line))
    return simplices

Reading the example filtration

In [89]:
simplices = read_filtration('filtration.txt')

Grouping by time

In [90]:
simplices.sort(key=lambda x: x.val)

Sorting the vertices

In [91]:
for s in simplices:
    s.vert.sort()

Creating dictionary for matrix creation

In [92]:
simplices_str =  [str(s.vert).strip('[]') for s in simplices]

d = {}
for i, s in enumerate(simplices_str):
    d[s] = i

Creating dictionary for final output

In [93]:
sDict = {}
for i, s in enumerate(simplices):
    sDict[i] = s

Creating dense matrix (dá pra melhorar pra calcular direto a sparse matrix, tem que melhorar)

In [94]:
s_matrix = np.zeros((len(simplices),len(simplices)))

for i, s in enumerate(simplices):
    if s.dim>0:
        vs = list(s.vert)
        for v in vs:
            temp = list(vs)
            temp.remove(v)
            #print temp
            s_matrix[d[str(temp).strip('[]')]][i] = 1

Transforming to sparse

In [95]:
N = 10

def nonnull(vector):
    answer = []
    for i,val in enumerate(vector):
        if val == 1:
            answer.append(i)
    return answer

sparse = [nonnull(s_matrix[:,i]) for i in range(N)]

Printing sparse matrix

In [96]:
print sparse

[[], [], [], [0, 1], [0, 2], [], [1, 5], [2, 5], [1, 2], [6, 7, 8]]


Reducing Matrix

In [97]:
N = M

lowsparse = [float('inf')]*N

def diffcols(col1,col2): #maybe we can optimise it, I couldn't find a way to do it directly in O(m) without transforming them into vector
    aux1 = np.array([0]*N)
    aux2 = np.array([0]*N)
    for i in range(len(col1)):
        aux1[col1[i]] = 1
    for i in range(len(col2)):
        aux2[col2[i]] = 1
    answer  = np.logical_xor(aux1,aux2).astype(int)
    answer = nonnull(answer)
    return answer
    
for j in range(N):
    if sparse[j]:
        lowcol = sparse[j][-1]
        lowsparse[j] = lowcol
        while lowcol in lowsparse[:j]:
            ind = lowsparse.index(lowcol)
            sparse[j] = diffcols(sparse[j],sparse[ind])
            if sparse[j]:
                lowcol = sparse[j][-1]
                lowsparse[j] = lowcol
            else:
                break
            

Printing reduced matrix

In [98]:
print sparse

[[], [], [], [0, 1], [0, 2], [], [1, 5], [], [], [6, 7, 8]]


Giving final answer

In [99]:
finalanswer = []
for j in range(N):
    if lowsparse[j] == float('inf'):
        if j not in lowsparse:
            aux = sDict[j]
            finalanswer.append(str(aux.dim)+' '+str(aux.val)+' '+'inf')
    else:
        aux1 = sDict[lowsparse[j]]
        aux2 = sDict[j]
        finalanswer.append(str(aux1.dim)+' '+str(aux1.val)+' '+str(aux2.val))
print finalanswer

['0 1.0 inf', '0 1.0 2.0', '0 1.0 2.0', '0 3.0 4.0', '0 1.0 4.0', '0 1.0 5.0', '1 5.0 6.0']
